In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import main as m
import regex as re

In [2]:
import selenium

In [3]:
# investigator 
# ablilities with embedid values not capturing correctly
# Complex deck reqs not being captured correctly
# some must includes not being captured correctly

In [4]:
invest, assets, events, skills = m.get_urls_by_type()


Getting investigator URL's...
Getting asset URL's...
Getting event URL's...
Getting skill URL's...


In [5]:
##########################################Helper functions#########################################################
def get_soup(url):
    '''
       Takes in a url for a card
       Returns html request result parsed using beautiful soup'''

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content)

    # locate urls on page and return
    return soup


def get_text_for_icon(text):

    # replace icon html with matching word in all caps
    icon_types = [
                  'action',
                  'reaction',
                  'wild', 
                  'willpower', 
                  'combat', 
                  'agility', 
                  'intellect', 
                  'wild',
                  'curse', 
                  'bless',
                  'rogue',
                  'survivor',
                  'seeker',
                  'guardian',
                  'mystic',
                  'neutral',
                  'skull',
                  'tablet',
                  'cultist',
                  'elder sign']
    
    for icon in icon_types:
    
        text = text.replace(f'<span class="icon-{icon}" title="{icon.capitalize()}"></span>', 
                                  f'{icon.upper()}')
        
        text = text.replace(f'<div class="card-text border-{icon}">\n<p>', '')

    text = text.replace(f'<span class="icon-wild" title="Any Skill"></span>', 'WILD')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Sign"></span>', 'ELDER_SIGN')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Thing"></span>', 'ELDER_THING')

    text = text.replace(f'<span class="icon-lightning" title="Fast Action"></span>', 'FAST ACTION')

    text = text.replace(f'<span class="icon-auto_fail" title="Auto Fail"></span>', 'TENTACLES')

    return text

def clean_html(text):

    # delete extraneous html
    dirt = [
            '</p>\n</div>',
            '</p>',
            '<p>',
            '<b>',
            '</b>',
            '<br/',
            '<1>',
            '<i>',
            '</i>',
            '><span>']
    
    for item in dirt:
        
        text = text.replace(item,'')

    return text

In [23]:
def get_xp(results):
    ''' Child of get_card_traits
        Takes in an BS object containing card information
        Returns XP cost of card if it exists
        Otherwise returns 0'''
    
    # locate xp info in object
    xp = results.find('div', class_='card-props')
    
    # if there is no value return 0
    if xp == None:
        
        return 0
    
    # otherwise return perform minor cleaning on value and return
    else:
        
        return re.search('[0-9]', str(xp)).group()
    
    
def get_icons(results):
    '''Child of get_card_traits
       Takes in request results for an arkhamdb page containing player card data
       Returns a string represintation of skill test icons on the card'''
      
    icons = ''

    # list containing each icon type
    icon_types = ['wild', 'willpower', 'combat', 'agility', 'intellect']

    # itterate through icon types
    for stat in icon_types:

        # get number of that icon on card from request results
        num_icons = len(results.find_all('span', class_=f'icon icon-{stat} color-{stat}'))

        # add that icon name to a string for each time it appears in request results
        for icon in range(num_icons):

            icons += f'{stat} '
            
    return icons.upper()[:-1]


#####find changes to find_all#######################################################################
def get_ability(results, faction):
    '''Child of get_card_traits
        Takes in request results for an arkhamdb page containing player card data
        Returns a string represintation of skill test icons on the card'''
    
    # gets html object and convert to string
    ability_string = str(results.find_all('div', class_=f'card-text border-{faction.lower()}'))
    
    # convert html to string, replace icons in text with string represintations
    ability_text = get_text_for_icon(ability_string)
    
    ability_text = clean_html(ability_text)
    
    return ability_text


def get_text_for_icon(text):

    # replace icon html with matching word in all caps
    icon_types = [
                  'action',
                  'reaction',
                  'wild', 
                  'willpower', 
                  'combat', 
                  'agility', 
                  'intellect', 
                  'wild',
                  'curse', 
                  'bless',
                  'rogue',
                  'survivor',
                  'seeker',
                  'guardian',
                  'mystic',
                  'neutral',
                  'skull',
                  'tablet',
                  'cultist',
                  'elder sign']
    
    for icon in icon_types:
    
        text = text.replace(f'<span class="icon-{icon}" title="{icon.capitalize()}"></span>', 
                                  f'{icon.upper()}')
        
        text = text.replace(f'<div class="card-text border-{icon}">\n<p>', '')

    text = text.replace(f'<span class="icon-wild" title="Any Skill"></span>', 'WILD')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Sign"></span>', 'ELDER_SIGN')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Thing"></span>', 'ELDER_THING')

    text = text.replace(f'<span class="icon-lightning" title="Fast Action"></span>', 'FAST ACTION')

    text = text.replace(f'<span class="icon-auto_fail" title="Auto Fail"></span>', 'TENTACLES')

    return text

def clean_html(text):

    # delete extraneous html
    dirt = [
            '</p>\n</div>',
            '</p>',
            '<p>',
            '<b>',
            '</b>',
            '<br/',
            '<1>',
            '<i>',
            '</i>',
            '><span>',
            '[',
            ']']
    
    for item in dirt:
        
        text = text.replace(item,'')

    return text


def get_cost_xp(results):
    
    cost_xp = str(results.find('div', class_='card-props'))

    cost = re.search(r'Cost:\s+(\d+)', cost_xp).group(1)

    xp = re.search(r'XP:\s+(\d+)', cost_xp).group(1)
    
    return cost, xp

In [8]:
# [
    
#  #title,
#  #sub_title,
#  #faction, 
#  #tipe, 
#  body_slot #### later
#  #traits,
#  cost,
#  #xp,
#  #icons,
#  #ability,
#  #artist,
#  #expansion,
#  #flavor

# ]

In [37]:
results = get_soup('https://arkhamdb.com/card/07305')

In [44]:
str(results.find('div', class_='card-props'))

'<div class="card-props">\n\t\tCost:     4.\n\t\t\tXP: 2.\n\t</div>'

In [87]:
def get_cost_xp(results):
    
    cost_xp = str(results.find('div', class_='card-props'))

    try:
    
        cost = re.search(r'Cost:\s+(\d+)', cost_xp).group(1)
        
    except:
        
        cost = 0

    try:
    
        xp = re.search(r'XP:\s+(\d+)', cost_xp).group(1)
        
    except:
        
        xp = 0
    
    return cost, xp


def get_subtitle(results):
    
    try:
        
        sub_title = results.find('div', class_='card-subname small').text.replace('\n', '').replace('\t', '')
        
    except:
        
        sub_title = ''
        
    return sub_title



def get_traits(results):
    
    try:
        
        traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')
        
    except:
        
        traits = ''
        
    return traits

In [88]:
re.search(r'XP:\s+(\d+)', cost_xp).group(1)

'2'

In [89]:
#  #title,
#  #sub_title,
#  #faction, 
#  #tipe, 
#  body_slot #### later
#  #traits,
#  cost,
#  #xp,
#  #icons,
#  #ability,
#  #artist,
#  #expansion,
#  #flavor

In [90]:
def get_event_traits(results):

    title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

    sub_title = get_subtitle(results)

    faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

    tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

    traits = get_traits(results)

    cost, xp = get_cost_xp(results)

    icons = get_icons(results)

    ability = get_ability(results, faction)

    artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

    flavor = results.find('div', class_='card-flavor small').text.replace('\n', '').replace('\t', '')

    expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '')

    return [title,
             sub_title,
             faction,
             tipe,
             traits,
             cost,
             xp,
             icons,
             ability,
             artist,
             expansion,
             flavor,
             url]

In [91]:
event_dict = {'title':[],
             'sub_title':[],
             'faction':[],
             'type':[],
             'traits':[],
             'cost':[],
             'XP':[],
             'icons':[],
             'ability':[],
             'artist':[],
             'expansion':[],
             'flavor':[],
             'url':[]}
    
# for each url get player card info from that page and add each element to skill_traits
for url in events:

    # make html request to arkham db and parse using BS
    results = get_soup(url)

    # get list of card elements card elements
    event_list = get_event_traits(results)

    event_list.append(url)

    print(f'Getting event card {event_list[0]}...')

    # itterate through card element titles and add each to a dictionary
    for i, key in enumerate(event_dict):

        event_dict[key].append(event_list[i])

print("Making dataframe...")

df = pd.DataFrame(event_dict)

Getting investigator card "Eat lead!"...
Getting investigator card "Eat lead!"...
Getting investigator card "Fool me once..."...
Getting investigator card "Get behind me!"...
Getting investigator card "Get over here!"...
Getting investigator card "Get over here!"...
Getting investigator card "Hit me!"...
Getting investigator card "I'll Pay You Back!"...
Getting investigator card "I'll see you in hell!"...
Getting investigator card "I'll take that!"...
Getting investigator card "I'm done runnin'!"...
Getting investigator card "I'm outta here!"...
Getting investigator card "I've got a plan!"...
Getting investigator card "I've got a plan!"...
Getting investigator card "I've had worse…"...
Getting investigator card "I've had worse…"...
Getting investigator card "I've had worse…"...
Getting investigator card "If it bleeds..."...
Getting investigator card "Let God sort them out..."...
Getting investigator card "Let me handle this!"...
Getting investigator card "Look what I found!"...
Getting

Getting investigator card Followed...
Getting investigator card Foresight...
Getting investigator card Forewarned...
Getting investigator card Fortuitous Discovery...
Getting investigator card Fortune or Fate...
Getting investigator card Friends in Low Places...
Getting investigator card Galvanize...
Getting investigator card Gang Up...
Getting investigator card Gaze of Ouraxsh...
Getting investigator card Ghastly Revelation...
Getting investigator card Glimpse the Unthinkable...
Getting investigator card Glimpse the Unthinkable...
Getting investigator card Glory...
Getting investigator card Grievous Wound...
Getting investigator card Grift...
Getting investigator card Grit Your Teeth...
Getting investigator card Guidance...
Getting investigator card Guidance...
Getting investigator card Guided by Faith...
Getting investigator card Hallow...
Getting investigator card Hand of Fate...
Getting investigator card Hand-Eye Coordination...
Getting investigator card Harmony Restored...
Getting

In [94]:
df.sub_title.value_counts()

           238
Yuggoth      1
Yoth         1
Name: sub_title, dtype: int64

In [66]:
title

'.25 Automatic'

In [67]:
sub_title

''

In [68]:
faction

'Rogue'

In [69]:
tipe

'Asset. '

In [70]:
traits

'Item. Weapon. Firearm. Illicit.'

In [71]:
xp

'2'

In [72]:
ability

'Fast. Uses (4 ammo).ACTION Spend 1 ammo: Fight. If the attacked enemy is exhausted, you get +2 COMBAT and deal +1 damage for this attack.REACTION After you evade an enemy at your location: Perform the above Fight ability without spending an action.'

In [73]:
flavor

''

In [74]:
artist

'Mauro Dal Bo'

In [75]:
expansion

'Into the Maelstrom #305'

In [76]:
icons

'COMBAT AGILITY'

In [96]:
for r in range(10):

    url = f'https://arkhamdb.com/find?q=t%3Aevent&decks=player{r}'

    # create request and soup objects
    try:
        
        html = requests.get(url)
        print('Page found')

    except:
        
        print('Done!!!!!!!!!')
        break

Page found
Page found
Page found
Page found
Page found
Page found
Page found
Page found
Page found
Page found


In [ ]:

for r in (10):

    url = f'https://arkhamdb.com/find?q=t%3Aevent&decks=player{num}'

# create request and soup objects
    try:
        
        html = requests.get(url)
        print('Page found')

    except:
        
        print('Done!!!!!!!!!')
        break
        
        
        
soup = BeautifulSoup(html.content, 'html.parser')

# locate urls on page
results = soup.find(id='list')

results = results.find_all('a', class_='card-tip')

# convert urls to string and make a list
results = [str(result['href']) for result in results]